## Задание 1
#### Импортируйте библиотеки pandas и numpy.
#### Загрузите "Boston House Prices dataset" из встроенных наборов данных библиотеки sklearn. Создайте датафреймы X и y из этих данных.

#### Разбейте эти датафреймы на тренировочные (X_train, y_train) и тестовые (X_test, y_test) с помощью функции train_test_split так, чтобы размер тестовой выборки составлял 30% от всех данных, при этом аргумент random_state должен быть равен 42.

#### Создайте модель линейной регрессии под названием lr с помощью класса LinearRegression из модуля sklearn.linear_model.

#### Обучите модель на тренировочных данных (используйте все признаки) и сделайте предсказание на тестовых.

#### Вычислите R2 полученных предказаний с помощью r2_score из модуля sklearn.metrics.


In [22]:
from sklearn import datasets
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [42]:
df_boston = datasets.load_boston()
X = pd.DataFrame(df_boston.data)
Y = pd.DataFrame(df_boston.target)

X_train, X_test, Y_train, Y_test =  train_test_split(X, Y, test_size = 0.3, random_state = 42)

lr = LinearRegression()
lr.fit(X_train, Y_train)
Y_pred = lr.predict(X_test)

Y_test['prediction'] = Y_pred
Y_test
#check_test = pd.DataFrame({
#    "y_test": Y_test,
#    "y_pred": Y_pred
#})
#print(type(df_boston))
#print(df_boston)

<ipython-input-42-c893d36b9320>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Y_test['prediction'] = Y_pred


,0,prediction
173,23.6,28.648960
274,32.4,36.495014
491,13.6,15.411193
72,22.8,25.403213
452,16.1,18.855280
...,...,...
441,17.1,17.403672
23,14.5,13.385941
225,50.0,39.983425
433,14.3,16.682863


In [48]:
from sklearn.metrics import r2_score

Y_test_data=Y_pred[0]
r2_score(Y_test[0], Y_pred)

0.7112260057484929

### Задание 2
#### Создайте модель под названием model с помощью RandomForestRegressor из модуля sklearn.ensemble.
#### Сделайте агрумент n_estimators равным 1000, max_depth должен быть равен 12 и random_state сделайте равным 42.
#### Обучите модель на тренировочных данных аналогично тому, как вы обучали модель LinearRegression, но при этом в метод fit вместо датафрейма y_train поставьте y_train.values[:, 0], чтобы получить из датафрейма одномерный массив Numpy, так как для класса RandomForestRegressor в данном методе для аргумента y предпочтительно применение массивов вместо датафрейма.
#### Сделайте предсказание на тестовых данных и посчитайте R2. Сравните с результатом из предыдущего задания.
#### Напишите в комментариях к коду, какая модель в данном случае работает лучше.


In [53]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=1000, max_depth=12, random_state=42)
model.fit(X_train, Y_train.values[:, 0])
model_pred = model.predict(X_test)

Y_test['model_pred'] = model_pred
r2_score(Y_test[0], model_pred)

<ipython-input-53-befdda1baaab>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Y_test['model_pred'] = model_pred


0.87472606157312

### *Задание 3
#### Вызовите документацию для класса RandomForestRegressor, найдите информацию об атрибуте feature_importances_. С помощью этого атрибута найдите сумму всех показателей важности, установите, какие два признака показывают наибольшую важность.


In [62]:
#help(RandomForestRegressor)
feature_imp = pd.Series(model.feature_importances_,index=X_train.columns).sort_values(ascending=False)
feature_imp
# 12 (blacks per town) и 5 (NOX) наиболее важные

12    0.415847
5     0.402682
7     0.063973
0     0.031676
10    0.018081
6     0.014299
4     0.014269
11    0.012451
9     0.011525
2     0.007138
8     0.005281
1     0.001543
3     0.001236
dtype: float64

### *Задание 4
#### В этом задании мы будем работать с датасетом, с которым мы уже знакомы по домашнему заданию по библиотеке Matplotlib, это датасет Credit Card Fraud Detection.Для этого датасета мы будем решать задачу классификации - будем определять,какие из транзакциции по кредитной карте являются мошенническими.Данный датасет сильно несбалансирован (так как случаи мошенничества относительно редки),так что применение метрики accuracy не принесет пользы и не поможет выбрать лучшую модель. Мы будем вычислять AUC, то есть площадь под кривой ROC.
#### Импортируйте из соответствующих модулей RandomForestClassifier, GridSearchCV и train_test_split.
#### Загрузите датасет creditcard.csv и создайте датафрейм df.
#### С помощью метода value_counts с аргументом normalize=True убедитесь в том, что выборка несбалансирована. Используя метод info, проверьте, все ли столбцы содержат числовые данные и нет ли в них пропусков. Примените следующую настройку, чтобы можно было просматривать все столбцы датафрейма:
#### pd.options.display.max_columns = 100.
#### Просмотрите первые 10 строк датафрейма df.

In [69]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, train_test_split
import pandas as pd

dataset = pd.read_csv('/Users/konstantinx/opt/creditcard.csv', delimiter=',')
print(dataset['Class'].value_counts(normalize=True))
print(dataset.info())
pd.options.display.max_columns = 100
print(dataset.head(10))


0    0.998273
1    0.001727
Name: Class, dtype: float64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float

#### Создайте датафрейм X из датафрейма df, исключив столбец Class.
#### Создайте объект Series под названием y из столбца Class.
#### Разбейте X и y на тренировочный и тестовый наборы данных при помощи функции train_test_split, используя аргументы: test_size=0.3, random_state=100, stratify=y.
#### У вас должны получиться объекты X_train, X_test, y_train и y_test.
#### Просмотрите информацию о их форме.


In [87]:
X = dataset.drop('Class', axis=1)
Y = dataset.loc[:,'Class']
type(Y)
X_train, X_test, Y_train, Y_test =  train_test_split(X, Y, test_size = 0.3, random_state = 42, stratify=Y)
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

(199364, 30) (85443, 30) (199364,) (85443,)


#### Для поиска по сетке параметров задайте такие параметры:
#### parameters = [{'n_estimators': [10, 15],
#### 'max_features': np.arange(3, 5),
#### 'max_depth': np.arange(4, 7)}]
#### Создайте модель GridSearchCV со следующими аргументами:
#### estimator=RandomForestClassifier(random_state=100),
#### param_grid=parameters,
#### scoring='roc_auc',
#### cv=3.


In [81]:
parameters = [{'n_estimators': [10, 15],
            'max_features': np.arange(3, 5),
            'max_depth': np.arange(4, 7)}]
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

model_grid = GridSearchCV(estimator=RandomForestClassifier(random_state=100),
                    param_grid=parameters, scoring='roc_auc', cv=3)


#### Обучите модель на тренировочном наборе данных (может занять несколько минут).
#### Просмотрите параметры лучшей модели с помощью атрибута best_params_.
#### Предскажите вероятности классов с помощью полученнной модели и метода predict_proba.
#### Из полученного результата (массив Numpy) выберите столбец с индексом 1 (вероятность класса 1) и запишите в массив y_pred_proba. Из модуля sklearn.metrics импортируйте метрику roc_auc_score.
#### Вычислите AUC на тестовых данных и сравните с результатом,полученным на тренировочных данных, используя в качестве аргументов массивы y_test и y_pred_proba.

In [82]:
model_grid.fit(X_train, Y_train)
model_grid.best_params_

{'max_depth': 6, 'max_features': 4, 'n_estimators': 15}

In [96]:
massive = model_grid.predict_proba(X_test)
y_pred_proba = massive[:, 1]
#print(Y_test.shape, y_pred_proba.shape)
from sklearn.metrics import roc_auc_score
roc_auc_score(Y_test, y_pred_proba)


(85443,) (85443,)


0.9574938646953419

### Доп.задача
#### 1). Загрузите датасет Wine из встроенных датасетов sklearn.datasets с помощью функции load_wine в переменную data.
#### 2). Полученный датасет не является датафреймом. Это структура данных, имеющая ключи аналогично словарю. Просмотрите тип данных этой структуры данных и создайте список data_keys, содержащий ее ключи.
#### 3). Просмотрите данные, описание и названия признаков в датасете. Описание нужно вывести в виде привычного, аккуратно оформленного текста, без обозначений переноса строки, но с самими переносами и т.д.



In [105]:
from sklearn.datasets import load_wine

data = load_wine()
print(type(data))
data_keys = data.keys()
print(data_keys)
print(data.DESCR)

<class 'sklearn.utils.Bunch'>
dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names'])
.. _wine_dataset:

Wine recognition dataset
------------------------

**Data Set Characteristics:**

    :Number of Instances: 178 (50 in each of three classes)
    :Number of Attributes: 13 numeric, predictive attributes and the class
    :Attribute Information:
 		- Alcohol
 		- Malic acid
 		- Ash
		- Alcalinity of ash  
 		- Magnesium
		- Total phenols
 		- Flavanoids
 		- Nonflavanoid phenols
 		- Proanthocyanins
		- Color intensity
 		- Hue
 		- OD280/OD315 of diluted wines
 		- Proline

    - class:
            - class_0
            - class_1
            - class_2
		
    :Summary Statistics:
    
    ============================= ==== ===== ======= =====
                                   Min   Max   Mean     SD
    ============================= ==== ===== ======= =====
    Alcohol:                      11.0  14.8    13.0   0.8
    Malic Acid:                   0.74  5.

#### 4). Сколько классов содержит целевая переменная датасета? Выведите названия классов.
#### 5). На основе данных датасета (они содержатся в двумерном массиве Numpy) и названий признаков создайте датафрейм под названием X.
#### 6). Выясните размер датафрейма X и установите, имеются ли в нем пропущенные значения.

In [125]:
#Class
print(data['target_names'])
print(data['frame'])
df_data = pd.DataFrame(data=data['data'], index=None, columns=data['feature_names'], dtype=None, copy=None)
print(df_data)
print(df_data.info())

['class_0' 'class_1' 'class_2']
None
     alcohol  malic_acid   ash  alcalinity_of_ash  magnesium  total_phenols  \
0      14.23        1.71  2.43               15.6      127.0           2.80   
1      13.20        1.78  2.14               11.2      100.0           2.65   
2      13.16        2.36  2.67               18.6      101.0           2.80   
3      14.37        1.95  2.50               16.8      113.0           3.85   
4      13.24        2.59  2.87               21.0      118.0           2.80   
..       ...         ...   ...                ...        ...            ...   
173    13.71        5.65  2.45               20.5       95.0           1.68   
174    13.40        3.91  2.48               23.0      102.0           1.80   
175    13.27        4.28  2.26               20.0      120.0           1.59   
176    13.17        2.59  2.37               20.0      120.0           1.65   
177    14.13        4.10  2.74               24.5       96.0           2.05   

     flavanoid

#### 7). Добавьте в датафрейм поле с классами вин в виде чисел, имеющих тип данных numpy.int64. Название поля - 'target'.
#### 8). Постройте матрицу корреляций для всех полей X. Дайте полученному датафрейму название X_corr.


In [129]:
df_data['target'] = data['target']
X_corr = df_data.corr()
X_corr

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
alcohol,1.000000,0.094397,0.211545,-0.310235,0.270798,0.289101,0.236815,-0.155929,0.136698,0.546364,-0.071747,0.072343,0.643720,-0.328222
malic_acid,0.094397,1.000000,0.164045,0.288500,-0.054575,-0.335167,-0.411007,0.292977,-0.220746,0.248985,-0.561296,-0.368710,-0.192011,0.437776
ash,0.211545,0.164045,1.000000,0.443367,0.286587,0.128980,0.115077,0.186230,0.009652,0.258887,-0.074667,0.003911,0.223626,-0.049643
alcalinity_of_ash,-0.310235,0.288500,0.443367,1.000000,-0.083333,-0.321113,-0.351370,0.361922,-0.197327,0.018732,-0.273955,-0.276769,-0.440597,0.517859
magnesium,0.270798,-0.054575,0.286587,-0.083333,1.000000,0.214401,0.195784,-0.256294,0.236441,0.199950,0.055398,0.066004,0.393351,-0.209179
total_phenols,0.289101,-0.335167,0.128980,-0.321113,0.214401,1.000000,0.864564,-0.449935,0.612413,-0.055136,0.433681,0.699949,0.498115,-0.719163
flavanoids,0.236815,-0.411007,0.115077,-0.351370,0.195784,0.864564,1.000000,-0.537900,0.652692,-0.172379,0.543479,0.787194,0.494193,-0.847498
nonflavanoid_phenols,-0.155929,0.292977,0.186230,0.361922,-0.256294,-0.449935,-0.537900,1.000000,-0.365845,0.139057,-0.262640,-0.503270,-0.311385,0.489109
proanthocyanins,0.136698,-0.220746,0.009652,-0.197327,0.236441,0.612413,0.652692,-0.365845,1.000000,-0.025250,0.295544,0.519067,0.330417,-0.499130
color_intensity,0.546364,0.248985,0.258887,0.018732,0.199950,-0.055136,-0.172379,0.139057,-0.025250,1.000000,-0.521813,-0.428815,0.316100,0.265668


#### 9). Создайте список high_corr из признаков, корреляция которых с полем target по абсолютному значению превышает 0.5 (причем, само поле target не должно входить в этот список).


In [138]:
high_corr = X_corr['target'][:13]
high_corr = high_corr[(high_corr > 0.5) | (high_corr < -0.5)]
high_corr

alcalinity_of_ash               0.517859
total_phenols                  -0.719163
flavanoids                     -0.847498
hue                            -0.617369
od280/od315_of_diluted_wines   -0.788230
proline                        -0.633717
Name: target, dtype: float64

#### 10). Удалите из датафрейма X поле с целевой переменной. Для всех признаков, названия которых содержатся в списке high_corr, вычислите квадрат их значений и добавьте в датафрейм X соответствующие поля с суффиксом '_2', добавленного к первоначальному названию признака. Итоговый датафрейм должен содержать все поля, которые, были в нем изначально, а также поля с признаками из списка high_corr, возведенными в квадрат. Выведите описание полей датафрейма X с помощью метода describe.

In [162]:
X2 = df_data.drop('target', axis=1)
names = list(high_corr.index)
names2 = [_.replace(_, _+'_2') for _ in names]
X2[names2] = X2[names]*X2[names]
X2.describe()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,alcalinity_of_ash_2,total_phenols_2,flavanoids_2,hue_2,od280/od315_of_diluted_wines_2,proline_2
count,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,1.780000e+02
mean,13.000618,2.336348,2.366517,19.494944,99.741573,2.295112,2.029270,0.361854,1.590899,5.058090,0.957449,2.611685,746.893258,391.142865,5.657030,5.110049,0.968661,7.322155,6.564591e+05
std,0.811827,1.117146,0.274344,3.339564,14.282484,0.625851,0.998859,0.124453,0.572359,2.318286,0.228572,0.709990,314.907474,133.671775,2.936294,4.211441,0.443798,3.584316,5.558591e+05
min,11.030000,0.740000,1.360000,10.600000,70.000000,0.980000,0.340000,0.130000,0.410000,1.280000,0.480000,1.270000,278.000000,112.360000,0.960400,0.115600,0.230400,1.612900,7.728400e+04
25%,12.362500,1.602500,2.210000,17.200000,88.000000,1.742500,1.205000,0.270000,1.250000,3.220000,0.782500,1.937500,500.500000,295.840000,3.036325,1.452100,0.612325,3.754075,2.505010e+05
50%,13.050000,1.865000,2.360000,19.500000,98.000000,2.355000,2.135000,0.340000,1.555000,4.690000,0.965000,2.780000,673.500000,380.250000,5.546050,4.558250,0.931250,7.728400,4.536045e+05
75%,13.677500,3.082500,2.557500,21.500000,107.000000,2.800000,2.875000,0.437500,1.950000,6.200000,1.120000,3.170000,985.000000,462.250000,7.840000,8.265700,1.254400,10.048900,9.702250e+05
max,14.830000,5.800000,3.230000,30.000000,162.000000,3.880000,5.080000,0.660000,3.580000,13.000000,1.710000,4.000000,1680.000000,900.000000,15.054400,25.806400,2.924100,16.000000,2.822400e+06
